In [702]:
import os
import fastf1 as ff1
import pandas as pd 
import numpy as np
import pandas as pd

Getting details from 2024 Bahrain Grand Prix

In [703]:
if not os.path.exists('./f1_cache'):
    os.makedirs('./f1_cache')
    ff1.Cache.enable_cache('./f1_cache')

session = ff1.get_session(2024, 'Bahrain', 'R')
session.load()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


In [704]:
laps2024 = session.laps[['Driver', 'LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']].copy()
laps2024.fillna(0, inplace=True)
# Convert columns to timedelta
laps2024['LapTime'] = pd.to_timedelta(laps2024['LapTime'])
laps2024['Sector1Time'] = pd.to_timedelta(laps2024['Sector1Time'])
laps2024['Sector2Time'] = pd.to_timedelta(laps2024['Sector2Time'])
laps2024['Sector3Time'] = pd.to_timedelta(laps2024['Sector3Time'])

# Convert timedelta to total seconds
laps2024['LapTime'] = laps2024['LapTime'].dt.total_seconds()
laps2024['Sector1Time'] = laps2024['Sector1Time'].dt.total_seconds()
laps2024['Sector2Time'] = laps2024['Sector2Time'].dt.total_seconds()
laps2024['Sector3Time'] = laps2024['Sector3Time'].dt.total_seconds()

sector_times_2024 = laps2024.groupby("Driver")[["Sector1Time", "Sector2Time", "Sector3Time"]].mean().reset_index()

sector_times_2024.head()

C:\Users\mayan\AppData\Local\Temp\ipykernel_3908\2382498084.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  laps2024.fillna(0, inplace=True)


,Driver,Sector1Time,Sector2Time,Sector3Time
0,ALB,31.311071,42.436839,24.129321
1,ALO,31.153825,42.104842,24.028702
2,BOT,30.432250,42.522339,24.249911
3,GAS,31.788571,42.159696,24.230589
4,HAM,30.972561,42.031789,23.836807


In [705]:
laps2024.head()

,Driver,LapTime,Sector1Time,Sector2Time,Sector3Time
0,VER,97.284,0.000,41.266,23.616
1,VER,96.296,30.916,41.661,23.719
2,VER,96.753,30.999,41.966,23.788
3,VER,96.647,30.931,41.892,23.824
4,VER,97.173,31.255,42.056,23.862


Getting details from 2025 Bahrain Qualifying

In [706]:
qualifying_2025 = pd.DataFrame({
    "Driver": ["Oscar Piastri", "George Russell", "Lando Norris", "Max Verstappen", "Lewis Hamilton",
               "Charles Leclerc", "Isack Hadjar", "Kimi Antonelli", "Yuki Tsunoda", "Alexander Albon",
               "Esteban Ocon", "Nico Hülkenberg", "Fernando Alonso", "Lance Stroll", "Carlos Sainz Jr.",
               "Pierre Gasly", "Oliver Bearman", "Jack Doohan", "Gabriel Bortoleto", "Liam Lawson"],
    "QualifyingTime (s)": [89.841, 90.009, 90.267, 90.423, 90.772,
                           90.175, 91.271, 90.213, 91.228, 91.706,
                           92.040, 92.067, 91.688, 92.283, 90.680 ,
                           90.216, 92.373, 91.245, 92.186, 92.165]
})

In [707]:
# Map full names to FastF1 3-letter codes
driver_mapping = {
    "Oscar Piastri": "PIA", "George Russell": "RUS", "Lando Norris": "NOR", "Max Verstappen": "VER",
    "Lewis Hamilton": "HAM", "Charles Leclerc": "LEC", "Isack Hadjar": "HAD", "Kimi Antonelli": "ANT",
    "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Esteban Ocon": "OCO", "Nico Hülkenberg": "HUL",
    "Fernando Alonso": "ALO", "Lance Stroll": "STR", "Carlos Sainz Jr.": "SAI", "Pierre Gasly": "GAS",
    "Oliver Bearman": "BEA", "Jack Doohan": "DOO", "Gabriel Bortoleto": "BOR", "Liam Lawson": "LAW"
}

In [708]:
qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)
qualifying_2025.head()

,Driver,QualifyingTime (s),DriverCode
0,Oscar Piastri,89.841,PIA
1,George Russell,90.009,RUS
2,Lando Norris,90.267,NOR
3,Max Verstappen,90.423,VER
4,Lewis Hamilton,90.772,HAM


In [709]:
merged_data = qualifying_2025.merge(sector_times_2024, left_on="DriverCode", right_on="Driver", how="left")

merged_data.head()

,Driver_x,QualifyingTime (s),DriverCode,Driver_y,Sector1Time,Sector2Time,Sector3Time
0,Oscar Piastri,89.841,PIA,PIA,31.113246,41.865140,23.969228
1,George Russell,90.009,RUS,RUS,31.061649,41.991596,23.755439
2,Lando Norris,90.267,NOR,NOR,31.001404,41.861333,23.954544
3,Max Verstappen,90.423,VER,VER,30.857000,41.414842,23.734123
4,Lewis Hamilton,90.772,HAM,HAM,30.972561,42.031789,23.836807


MODEL TIME BABY!!!

In [710]:
X = merged_data.drop(columns=['Driver_x', 'Driver_y', 'DriverCode']).fillna(0)
y = laps2024.groupby("Driver")["LapTime"].mean().reset_index()["LapTime"]

In [711]:
from sklearn.ensemble import RandomForestRegressor 
model = RandomForestRegressor(n_estimators=50)
model.fit(X, y)

RandomForestRegressor(n_estimators=50)

In [719]:
predicted_race_times = model.predict(X)
qualifying_2025["PredictedRaceTime (s)"] = predicted_race_times

# Rank drivers by predicted race time
qualifying_2025 = qualifying_2025.sort_values(by="PredictedRaceTime (s)",axis=0, ascending=True)

In [720]:
print("\n🏁 Prediction of 2025 Bahrain GP Winner 🏁\n")
print(qualifying_2025[["Driver", "PredictedRaceTime (s)"]])


🏁 Prediction of 2025 Bahrain GP Winner 🏁

               Driver  PredictedRaceTime (s)
0       Oscar Piastri              97.221918
11    Nico Hülkenberg              97.346865
2        Lando Norris              97.459514
3      Max Verstappen              97.463804
4      Lewis Hamilton              97.484750
13       Lance Stroll              97.621923
6        Isack Hadjar              97.680585
7      Kimi Antonelli              97.686699
8        Yuki Tsunoda              97.748915
19        Liam Lawson              97.760046
10       Esteban Ocon              97.895934
9     Alexander Albon              98.091104
12    Fernando Alonso              98.109956
17        Jack Doohan              98.135793
1      George Russell              98.144537
15       Pierre Gasly              98.276185
16     Oliver Bearman              98.277488
5     Charles Leclerc              98.286547
18  Gabriel Bortoleto              98.310416
14   Carlos Sainz Jr.              98.352516
